In [1]:
import numpy as np
from astropy.io import fits
import matplotlib
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import os
from ProjectF import MLAData,classification, Object,storing,Rebin
import random

In [2]:

TrainingDir = os.path.normpath("D:\Data")
slash =  os.path.normpath("\\")
TrainingFolder =  os.path.normpath("\Training")

In [3]:
PlateDir ="D:"+slash+"Data"+slash+"Full5.txt"
with open(PlateDir) as f:
    Spectra_Files = f.read().splitlines()

In [4]:
Spectra_Files

['4090.fits',
 '5177.fits',
 '6395.fits',
 '7380.fits',
 '4176.fits',
 '5208.fits',
 '6407.fits',
 '7420.fits',
 '4177.fits',
 '5300.fits',
 '6461.fits',
 '7429.fits',
 '4183.fits',
 '5346.fits',
 '6482.fits',
 '4196.fits',
 '5364.fits',
 '6514.fits',
 '4205.fits',
 '5408.fits',
 '6588.fits',
 '4214.fits',
 '5482.fits',
 '6593.fits']

0 = flux
1 = class
2 = or
3 = and
4 = inv
5 = red

In [5]:
wav_log=[]
Full_table = []
PLATEIDs = []
for spectrum in Spectra_Files:
    plate_ = fits.open( TrainingDir+slash+ spectrum ,memmap=True)
    primhdu_ = plate_[0]
    hdu =  plate_[1].data
    Full_table.append(hdu)
    PLATEIDs.append(primhdu_.header['Plate'])
    wav_log.append(primhdu_.header['LogWav'])

In [6]:
PLATEIDs

[4090,
 5177,
 6395,
 7380,
 4176,
 5208,
 6407,
 7420,
 4177,
 5300,
 6461,
 7429,
 4183,
 5346,
 6482,
 4196,
 5364,
 6514,
 4205,
 5408,
 6588,
 4214,
 5482,
 6593]

In [7]:
p = 0
rebin = []
weights = []
Bin_Size=10
while p < len(Full_table):
    current_hdu = Full_table[p]
    r, w = Rebin(current_hdu, Bin_Size )    
    rebin.append(r)
    weights.append(w)
    p=p+1

In [8]:
All_z  = []
All_n  = []
All_c  = []
i = 0
while i < len(PLATEIDs):
    plate_r  = []
    plate_n  = []
    plate_c  = []
    plate_hdu  = Full_table[i]
    obj = 0
    while obj <len(plate_hdu):
        currentobj = plate_hdu[obj]
        Currentplate_z = currentobj[4]
        Currentplate_n = currentobj[5]
        Currentplate_c= currentobj[1]
        plate_r.append(Currentplate_z)
        plate_n.append(Currentplate_n)
        plate_c.append(Currentplate_c)
        obj = obj +1
    All_z.append(plate_r)
    All_n.append(plate_n)
    All_c.append(plate_c)
    i = i+1



In [9]:
i=0

while i <len(PLATEIDs):
    C_Plate = PLATEIDs[i]
    plate_z  = All_z[i]
    plate_n  = All_n[i]
    plate_c  = All_c[i]
    Current_wav = wav_log[i]
    obj=0
    col1 = fits.Column(name='Name', format='20A', array=np.array(plate_n))
    col2 = fits.Column(name='Rebin_Flux', format='PD()', array=np.array(rebin[i],dtype=np.object))
    col3 = fits.Column(name='Class', format='I', array=np.array(plate_c))
    col4 = fits.Column(name='Weight', format='PD()', array=np.array(weights[i],dtype=np.object))
    col5 = fits.Column(name='Redshift', format='D', array=np.array(plate_z))    
    cols = fits.ColDefs([col1, col2,col3, col4,col5])
    tbhdu = fits.BinTableHDU.from_columns(cols)
    prihdr = fits.Header()
    prihdr['Plate'] = C_Plate
    prihdr['LogWav'] = Current_wav
    prihdu = fits.PrimaryHDU(header=prihdr)
    file_name = TrainingDir+slash+"rebin"+slash+np.str(C_Plate)+'ReBin.fits'
    thdulist = fits.HDUList([prihdu, tbhdu])
    thdulist.writeto(file_name)
    print(file_name)
    i=i+1

D:\Data\rebin\4090ReBin.fits
D:\Data\rebin\5177ReBin.fits
D:\Data\rebin\6395ReBin.fits
D:\Data\rebin\7380ReBin.fits
D:\Data\rebin\4176ReBin.fits
D:\Data\rebin\5208ReBin.fits
D:\Data\rebin\6407ReBin.fits
D:\Data\rebin\7420ReBin.fits
D:\Data\rebin\4177ReBin.fits
D:\Data\rebin\5300ReBin.fits
D:\Data\rebin\6461ReBin.fits
D:\Data\rebin\7429ReBin.fits
D:\Data\rebin\4183ReBin.fits
D:\Data\rebin\5346ReBin.fits
D:\Data\rebin\6482ReBin.fits
D:\Data\rebin\4196ReBin.fits
D:\Data\rebin\5364ReBin.fits
D:\Data\rebin\6514ReBin.fits
D:\Data\rebin\4205ReBin.fits
D:\Data\rebin\5408ReBin.fits
D:\Data\rebin\6588ReBin.fits
D:\Data\rebin\4214ReBin.fits
D:\Data\rebin\5482ReBin.fits
D:\Data\rebin\6593ReBin.fits
